In [1]:
import string
from nltk.corpus import stopwords
import nltk 
import pandas as pd
import numpy as np
import re
from collections import Counter
import csv
from nltk import stem

In [2]:
#nltk.download_shell() 

In [3]:
from nltk.corpus import stopwords

# Dictionary

In [9]:
abb = pd.read_csv("abb_new.csv")
abb =abb.drop_duplicates()
abb =abb.dropna()
abb.to_csv("abb_01_05.csv")
print(abb.head())

                              Name  Form
0               Above Ground Level   AGL
1           Actual Time Of Arrival   ATA
2         Actual Time Of Departure   ATD
3                     Aeronautical  AERO
4  Aeronautical Information Manual   AIM


In [10]:
print(len(abb))
print(len(abb['Form'].unique()))

465
456


In [11]:
abb['Form'].value_counts().head(10)
#some rows have the same column value

ATA     2
RWY     2
CAT     2
FSS     2
Z       2
EST     2
N       2
AWOS    2
TCAS    2
ISDU    1
Name: Form, dtype: int64

In [12]:
array = ['FSS','EST','CAT','AWOS','RWY','Z','ATA','TCAS','N']
abb.loc[abb['Form'].isin(array)]

,Name,Form
1,Actual Time Of Arrival,ATA
13,Air Traffic Area,ATA
40,Automated Surface Observation Sys (Or Weather ...,AWOS
46,Automatic Weather Observing/Reporting System,AWOS
65,Category,CAT
79,Clear Air Turbulence,CAT
97,Coordinated Universal Time,Z
127,Eastern Standard Time,EST
139,"Estimate, Estimation",EST
178,Flight Service Station,FSS


In [13]:
#print(abb[abb['Form']=='AWOS'].iloc[0,0])
#print(abb[abb['Form']=='AWOS'].iloc[1,0])

In [14]:
array = ['FSS','EST','CAT','AWOS','RWY','Z','ATA','TCAS','N']
abb.loc[abb['Form'].isin(array)]

,Name,Form
1,Actual Time Of Arrival,ATA
13,Air Traffic Area,ATA
40,Automated Surface Observation Sys (Or Weather ...,AWOS
46,Automatic Weather Observing/Reporting System,AWOS
65,Category,CAT
79,Clear Air Turbulence,CAT
97,Coordinated Universal Time,Z
127,Eastern Standard Time,EST
139,"Estimate, Estimation",EST
178,Flight Service Station,FSS


In [15]:
#Unify some values
abb.loc[1,'Name'] = 'Actual Time Of Arrival/Air Traffic Area'
abb.loc[13,'Name'] = 'Actual Time Of Arrival/Air Traffic Area'
abb.loc[40,'Name'] = 'Automatic Weather Observing System'
abb.loc[46,'Name'] = 'Automatic Weather Observing System'
abb.loc[65,'Name'] ='Category/Clear Air Turbulence'
abb.loc[79,'Name'] ='Category/Clear Air Turbulence'
abb.loc[950,'Name'] = 'Coordinated Universal Time'
abb.loc[127,'Name'] = 'Eastern Standard Time/Estimate'
abb.loc[139,'Name'] = 'Eastern Standard Time/Estimate'
abb.loc[179,'Name'] = 'Flight Service Station'
abb.loc[802,'Name'] = 'North'
abb.loc[386,'Name'] = 'Runway(s)'
abb.loc[900,'Name'] = 'Traffic Alert and Collision Avoidance'

In [16]:
abb =abb.drop_duplicates()
abb =abb.dropna()
abb['Form'].value_counts().head(10)

USA     1
TA      1
HELI    1
RPM     1
LTR     1
DFDR    1
CHK     1
TOGW    1
MSL     1
ISDU    1
Name: Form, dtype: int64

In [17]:
#Remove rows that contain numeric values
indexNames = abb[(abb['Form'].str.contains('0')) | (abb['Form'].str.contains('1'))|
                 (abb['Form'].str.contains('2')) | (abb['Form'].str.contains('3'))|
                 (abb['Form'].str.contains('4')) | (abb['Form'].str.contains('5'))|
                 (abb['Form'].str.contains('6')) | (abb['Form'].str.contains('7'))|
                 (abb['Form'].str.contains('8')) | (abb['Form'].str.contains('9'))].index
abb.drop(indexNames , inplace=True)

In [18]:
print(len(abb))
print(len(abb['Form'].unique()))

454
454


In [19]:
#Create the dictionary
dict_abb = {}
for i in range(len(abb)):   
    word = abb.iloc[i,1]
    value = abb.iloc[i,0]
    dict_abb[word] = value
#Standard 2400' Hi-Intensity Approach Lighting System With Sequenced Flashers (Cat I Config), ALSF-1
#Standard 2400' Hi-Intensity Approach Lighting System With Sequenced Flashers (Cat Ii Config), ALSF-2
{k.lower(): v.lower() for k, v in dict_abb.items()}

{'agl': 'above ground level',
 'ata': 'actual time of arrival/air traffic area',
 'atd': 'actual time of departure',
 'aero': 'aeronautical',
 'aim': 'aeronautical information manual',
 'acr': 'air carrier',
 'adc': 'air data computer',
 'adiz': 'air defense identification zone',
 'adg': 'air driven generator',
 'afb': 'air force base',
 'ang': 'air national guard',
 'arsr': 'air route surveillance radar',
 'artcc': 'air route traffic control center',
 'atc': 'air traffic control',
 'acft': 'aircraft',
 'atp': 'airline transport pilot certificate',
 'atr': 'airline transport rating',
 'arpt': 'airport',
 'afd': 'airport/facility directory',
 'asr': 'airport surveillance radar',
 'atct': 'airport traffic control tower',
 'atog': 'allowable takeoff gross weight',
 'ac': 'alternating current',
 'alt': 'altitude',
 'altdev': 'altitude deviation',
 'amb': 'ambiguity',
 'am': 'ante meridiem',
 'apch': 'approach',
 'apch ctl': 'approach control',
 'approx': 'approximately',
 'rnav': 'area nav

# ASRS

In [20]:
df = pd.read_csv('asrs_v1_1112.csv')
df = df.head(2000)
re1 = df['report 1']

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (15,19,20,28,33,44,51,53,54,55,56,66,91) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
def text_process1(mess):
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc).lower()
    return nopunc
re1_new =re1.apply(text_process1)

In [22]:
#replace abbreviation with words
def replace_words(s, words):
    for k, v in dict.items():
        s = s.replace(k, v)
    return s

In [23]:
len(re1_new)

2000

In [24]:
for i in range(len(re1_new)):
    re1_new[i] =replace_words(re1_new[i], dict_abb)

In [25]:
result = pd.concat([df, re1_new], axis=1, sort=False)
result.head(2)

,Unnamed: 0,index,acn,local_time,airport,state,radial,distance,agl,msl,...,month,airspace_class_1_1,airspace_abre_1_1,airspace_class_1_2,airspace_abre_1_2,airspace_class_2_1,airspace_abre_2_1,airspace_class_2_2,airspace_abre_2_2,report 1
0,0,0,622599,1201-1800,N45.Airport,NY,NaN,2.0,NaN,9000.0,...,7,E,ZBW,NaN,NaN,NaN,NaN,NaN,NaN,today flying at 9000 feet near msv on an instr...
1,1,1,622789,0601-1200,BRV.VORTAC,VA,NaN,15.0,NaN,2000.0,...,7,E,PCT,NaN,NaN,NaN,NaN,NaN,NaN,inadvertent penetration of r6601 fort ap hill ...


In [26]:
#result.to_csv('asrs_v1_0105.csv')

In [27]:
#df['report 1'][0].lower()

In [28]:
re1_new[0]

'today flying at 9000 feet near msv on an instrument flight rules flight plan i saw 2 parachuters drop past within 100 feet the first at my 11 oclock pos btwn 50 feet and 100 feet away saw that one but didnt comprehend what it was went by too fast i thought it was some kind of collapsed balloon at first then another at my 2303 oclock pos less than 50 feet off my wing just afeeter that atc told me to turn to an immediate 160 deg hdg i had been on a 90 deg hdg i told them what i had just seen and he said yeah there was a jump acfeet above me only mention of para jumping in the fss briefing was nw of wilkesbarre and atc hadnt said a peep before the incident i was in and out of cumulus at that alt and had only been out of a cloud for about 30 seconds before the jumpers passed through my alt too fast for me to attempt any evasive action their parachutes were not deployed but a small parachute or an unreleased parachute trailed each by a few feet the jumpers were close enough to see clrly th

In [29]:
#Combine all lists to one list
re1_word=[]
for i in range(len(re1_new)):
    re1_new[i] = re1_new[i].split(' ')
    re1_word = re1_word +re1_new[i]

In [30]:
nltk.download('wordnet')
wnl = stem.WordNetLemmatizer()
porter = stem.porter.PorterStemmer()

[nltk_data] Downloading package wordnet to /Users/mac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [31]:
#Remove stop words from the list
stop_words = stopwords.words('english')
re1_word =[word for word in re1_word if word not in stop_words]
#Stemming
#re1_word = [stem(word) for word in re1_word]
re1_word = [wnl.lemmatize(word) for word in re1_word]
#remove words contain number
re1_word = [word for word in re1_word if all(digit not in word for digit in "1234567890")]

In [32]:
re1_word

['today',
 'flying',
 'foot',
 'near',
 'msv',
 'instrument',
 'flight',
 'rule',
 'flight',
 'plan',
 'saw',
 'parachuter',
 'drop',
 'past',
 'within',
 'foot',
 'first',
 'oclock',
 'po',
 'btwn',
 'foot',
 'foot',
 'away',
 'saw',
 'one',
 'didnt',
 'comprehend',
 'went',
 'fast',
 'thought',
 'kind',
 'collapsed',
 'balloon',
 'first',
 'another',
 'oclock',
 'po',
 'le',
 'foot',
 'wing',
 'afeeter',
 'atc',
 'told',
 'turn',
 'immediate',
 'deg',
 'hdg',
 'deg',
 'hdg',
 'told',
 'seen',
 'said',
 'yeah',
 'jump',
 'acfeet',
 'mention',
 'para',
 'jumping',
 'f',
 'briefing',
 'nw',
 'wilkesbarre',
 'atc',
 'hadnt',
 'said',
 'peep',
 'incident',
 'cumulus',
 'alt',
 'cloud',
 'second',
 'jumper',
 'passed',
 'alt',
 'fast',
 'attempt',
 'evasive',
 'action',
 'parachute',
 'deployed',
 'small',
 'parachute',
 'unreleased',
 'parachute',
 'trailed',
 'foot',
 'jumper',
 'close',
 'enough',
 'see',
 'clrly',
 'wearing',
 'helmet',
 'visor',
 'took',
 'good',
 'hr',
 'calm',
 'sky

In [33]:
counts = Counter(re1_word)
frequency =[(l,k) for k,l in sorted([(j,i) for i,j in counts.items()], reverse=True)]

In [36]:
frequency[0:10]

[('rwy', 4777),
 ('acfeet', 4326),
 ('foot', 3625),
 ('flight', 3292),
 ('apch', 2226),
 ('afeeter', 1951),
 ('time', 1913),
 ('twr', 1832),
 ('would', 1691),
 ('lndg', 1582)]

In [35]:
with open('word_count.csv', "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(frequency)